This notebook merges the RecipePuppy graph to the KitchenConcepts+Affordance+CN graph.

In [1]:
from rdflib import Graph, Namespace, RDF, URIRef, RDFS
from rdflib.namespace import OWL, SKOS

In [2]:
def create_namespace(graph, namespace, prefix):
    """
    Binds a namespace to a given graph.
    Args:
        - graph: graph object
        - namespace: uri of the namespace to be bound to the graph
        - prefix: the prefix of the given namespace
    Output: namespace instance bound to a specified graph
    """
    ns = Namespace(namespace)
    graph.namespace_manager.bind(prefix, namespace)

    return ns

In [3]:
# obtain the graphs from the turtle files
aff_bft_cn_file = './graphs/aff_bft_cn.ttl'
aff_bft_cn_g = Graph()
recipe_file = '../data/RecipePuppy/recipe_graph.ttl'
recipe_g = Graph()
aff_bft_cn_g.parse(aff_bft_cn_file, format= 'turtle')
recipe_g.parse(recipe_file, format= 'turtle')

<Graph identifier=Ne5fa826fa6714b6c96f3a834b85167e8 (<class 'rdflib.graph.Graph'>)>

In [4]:
# combine graphs
merged_g = Graph()
merged_g = aff_bft_cn_g + recipe_g
len(merged_g)

6126

In [5]:
# remove unnecessary owl ontology triple
merged_g.remove((URIRef('http://test.org/recipe.owl'),RDF.type,OWL.Ontology))
merged_g.remove((URIRef('http://test.org/cn.owl'),RDF.type,OWL.Ontology))

In [6]:
rp_ns = create_namespace(merged_g, 'http://test.org/recipe_puppy#', 'rp')
bft_ns = create_namespace(merged_g, 'http://test.org/bft.owl#', 'bft')
rp_owl = create_namespace(merged_g, "http://test.org/recipe.owl#", 'rcp')

In [7]:
instances = ['milk', 'apple', 'lemon', 'coffee', 'croissant', 'garnish', 'egg', 'orange_juice', 'apple_juice', 'butter']

In [8]:
ingredients_q = merged_g.query(
"""
prefix bft: <http://test.org/bft.owl#> 
prefix rp: <http://test.org/recipe_puppy#> 

SELECT distinct ?obj WHERE {
  ?sub rp:ingredient ?obj .
}

""")

In [9]:
c = 0
for i in instances:
    for r in ingredients_q:
        ingredient = r['obj'][29:]
        if i == ingredient:
            c+=1
            merged_g.add( (URIRef(f'http://test.org/bft.owl#{i}'), OWL.sameAs, rp_ns[ingredient]) )
print(c)

7


In [10]:
merged_g.add( (rp_owl.Ingredient, RDFS.subClassOf, bft_ns.Food) )

In [11]:
merged_g.serialize('./graphs/aff_bft_cn_recipe.ttl', format='turtle')